### Packages

In [1]:
from sklearn.datasets import load_breast_cancer
import numpy as np
import random
import diffprivlib
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from diffprivlib.models import LogisticRegression as DP_LogisticRegression
from diffprivlib.models import GaussianNB as DPGaussianNB  
from diffprivlib.models import DecisionTreeClassifier as DP_DecisionTreeClassifier
from diffprivlib.models import RandomForestClassifier as DP_RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
from sklearn.preprocessing import LabelEncoder
import math
from scipy.special import softmax
from sklearn.datasets import make_classification
import scipy.stats as stats
import time
import pickle
import os

### Nonprivate models

In [2]:
def get_nonprivate_model(model_name, random_state):
    

    if model_name == "Random_forest":
        model= RandomForestClassifier(random_state=random_state)
        
        
    elif model_name == "Decision_tree":
        model=DecisionTreeClassifier()
    
        
    elif model_name == "Naive_bayes":
       model=GaussianNB()
        
    elif model_name == "Logistic_regression":
        model=LogisticRegression(max_iter=5000, random_state=random_state)
    return model

### Private models

In [3]:
def get_private_model(model_name, X_train, y_train, y, epsilon):
    EPSILON =epsilon
  
    classes = np.unique(y)

    if model_name == "Random_forest":
        # Define bounds for each feature (for differential privacy)
        min_bounds = -20
        max_bounds = 20
        bounds = (min_bounds, max_bounds)
        
        # Initialize private Random Forest with differential privacy
        model = DP_RandomForestClassifier(
            epsilon=EPSILON,    # Differential privacy parameter
            n_estimators=127,   # Number of trees in the forest
            max_depth=7,        # Maximum depth of the trees
            bounds=bounds,      # Data bounds for differential privacy
            random_state=42,
            classes=classes
        )
    elif model_name == "Decision_tree":
        # Define bounds for each feature (for differential privacy)
        min_bounds = -20
        max_bounds = 20
        bounds = (min_bounds, max_bounds)
    
        # Initialize private Random Forest with differential privacy
        model = DP_DecisionTreeClassifier(
            epsilon=EPSILON,    # Differential privacy parameter
            max_depth=6,        # Maximum depth of the trees
            bounds=bounds,      # Data bounds for differential privacy
            random_state=42,
            classes=classes
        )
    elif model_name == "Naive_bayes":
        # Define bounds for each feature (for differential privacy)
        min_bounds =-20 
        max_bounds =20 
        bounds = (min_bounds, max_bounds)
        
        # Initialize private Naive Bayes with differential privacy
        model = DPGaussianNB(
            epsilon=EPSILON,    # Differential privacy parameter
            bounds=bounds,      # Data bounds for differential privacy
            random_state=42
        )
    elif model_name == "Logistic_regression":
        # Calculate the data norm (L2 norm)
        data_norm = 20
        
        # Initialize private Logistic Regression
        model = DP_LogisticRegression(
            epsilon=EPSILON,
            data_norm=data_norm,
            random_state=42
        )
    return model    

### Helper functions and PrivQuant algorithm for PCOQS method

In [4]:
def NoisyRC(range_bounds, D, sigma):
    """
    Noisy Range Count for float values with Gaussian noise.

    Parameters:
    range_bounds (tuple): A tuple (a, b) representing the range [a, b].
    D (list): The sorted dataset.
    sigma (float): The standard deviation of the Gaussian noise.

    Returns:
    int: The noisy count of elements in the range [a, b].
    """
    a, b = range_bounds
    count = sum(1 for z in D if a <= z <= b)
    noise = np.random.normal(0, sigma)
    noisy_count = count + noise
    return max(0, int(np.floor(noisy_count)))  # Ensure non-negative count


def PrivQuant(D, alpha, rho, lower_bound=0, upper_bound=1, delta=1e-10):
    """
    Differentially Private Quantile Approximation Algorithm without integer conversion.

    Parameters:
    D (list): The sorted dataset.
    alpha (float): The quantile level (e.g., 0.5 for median).
    rho (float): The privacy parameter (smaller = more private).
    lower_bound (float): Lower bound of the search space.
    upper_bound (float): Upper bound of the search space.
    delta (float): Small positive value to ensure convergence.

    Returns:
    float: A differentially private approximation of the quantile x_{(m)}.
    """

    
    n = len(D)
    max_iterations = int(np.ceil(np.log2((upper_bound - lower_bound) / delta)))
    sigma = np.sqrt(max_iterations / (2 * rho))  # Noise scale for Gaussian mechanism
    m = int(np.ceil((1 - alpha) * (n + 1)))

    left, right = lower_bound, upper_bound

    for i in range(max_iterations):
        mid = (left + right) / 2
        c = NoisyRC((lower_bound, mid), D, sigma)
        
        if c < m:
            left = mid + delta
        else:
            right = mid

    return np.round((left + right) / 2, 2)



### Helper functions and Angelopoulos et al. algorithm

In [5]:
# Optimal gamma is a root.
def get_optimal_gamma(scores,n,alpha,m,epsilon):
    a = alpha**2
    b = - ( alpha*epsilon*(n+1)*(1-alpha)/2 + 2*alpha )
    c = 1
    best_q = 1
    gamma1 = (-b + np.sqrt(b**2 - 4*a*c))/(2*a)
    gamma2 = (-b - np.sqrt(b**2 - 4*a*c))/(2*a)

    gamma1 = min(max(gamma1,1e-12),1-1e-12)
    gamma2 = min(max(gamma2,1e-12),1-1e-12)

    bins = np.linspace(0,1,m)

    q1 = get_private_quantile(scores, alpha, epsilon, gamma1, bins)
    q2 = get_private_quantile(scores, alpha, epsilon, gamma2, bins)

    return (gamma1, q1) if q1 < q2 else (gamma2, q2)

def get_optimal_gamma_m(n, alpha, epsilon):
    candidates_m = np.logspace(4,6,50).astype(int)
    scores = np.random.rand(n,1)
    best_m = int(1/alpha)
    best_gamma = 1
    best_q = 1
    for m in candidates_m:
        gamma, q = get_optimal_gamma(scores,n,alpha,m,epsilon)
        if q < best_q:
            best_q = q
            best_m = m
            best_gamma = gamma
    return best_m, best_gamma

def get_private_quantile(scores, alpha, epsilon, gamma, bins):
    n = scores.shape[0]
    epsilon_normed = epsilon*min(alpha, 1-alpha)
    # Get the quantile
    qtilde = get_qtilde(n, alpha, gamma, epsilon, bins.shape[0])
    scores = scores.squeeze()
    score_to_bin = np.digitize(scores,bins)
    binned_scores = bins[np.minimum(score_to_bin,bins.shape[0]-1)]
    w1 = np.digitize(binned_scores, bins)
    w2 = np.digitize(binned_scores, bins, right=True)
    # Clip bins
    w1 = np.maximum(np.minimum(w1,bins.shape[0]-1),0)
    w2 = np.maximum(np.minimum(w2,bins.shape[0]-1),0)
    lower_mass = np.bincount(w1,minlength=bins.shape[0]).cumsum()/qtilde
    upper_mass = (n-np.bincount(w2,minlength=bins.shape[0]).cumsum())/(1-qtilde)
    w = np.maximum( lower_mass , upper_mass )
    sampling_probabilities = softmax(-(epsilon_normed/2)*w)
    # Check
    sampling_probabilities = sampling_probabilities/sampling_probabilities.sum()
    qhat = np.random.choice(bins,p=sampling_probabilities)
    return qhat

def get_shat_from_scores_private(scores, alpha, epsilon, gamma, score_bins):
    shat = get_private_quantile(scores, alpha, epsilon, gamma, score_bins)
    return shat

def get_qtilde(n,alpha,gamma,epsilon,m):
    qtilde = (n+1)*(1-alpha)/(n*(1-gamma*alpha))+2/(epsilon*n)*np.log(m/(gamma*alpha))
    qtilde = min(qtilde, 1-1e-12)
    return qtilde

### Data simulation function

In [6]:
def simulate_normal_classification(
    n_samples_per_class,
    n_features,
    n_classes,
    class_means=None,
    class_covariances=None,
    random_state=None
):
    """
    Simulate classification data directly from Normal distributions.
    
    Parameters:
        n_samples_per_class (int): Number of samples per class.
        n_features (int): Number of features.
        n_classes (int): Number of classes.
        class_means (list of arrays): List of mean vectors for each class. If None, generated randomly.
        class_covariances (list of arrays): List of covariance matrices for each class. If None, identity matrices are used.
        random_state (int): Seed for reproducibility.
    
    Returns:
        X (ndarray): Feature matrix of shape (n_samples, n_features), rounded to 4 decimal places.
        y (ndarray): Class labels of shape (n_samples,), rounded to 4 decimal places.
    """
    np.random.seed(random_state)
    
    X = []
    y = []
    
    # Generate means and covariances if not provided
    if class_means is None:
        class_means = [np.random.uniform(-5, 5, n_features) for _ in range(n_classes)]
    if class_covariances is None:
        class_covariances = [np.eye(n_features) for _ in range(n_classes)]
    
    for class_idx in range(n_classes):
        # Draw samples from the Normal distribution
        samples = np.random.multivariate_normal(
            mean=class_means[class_idx],
            cov=class_covariances[class_idx],
            size=n_samples_per_class
        )
        X.append(samples)
        y.extend([class_idx] * n_samples_per_class)
    
    # Combine data and shuffle
    X = np.vstack(X)
    y = np.array(y)
    indices = np.random.permutation(len(y))
    
    # Round X and y to 4 decimal places
    X = np.round(X[indices], 4)
    y = np.round(y[indices], 4)
    
    return X, y


### Simulation function

In [7]:


def run_simulation(
    private_model,
    private_conformal,
    model_name,
    n_samples_per_class,
    n_features,
    n_classes,
    class_means,
    class_covariances,
    alpha,
    epsilon,
    conformal_epsilon,
    n_trials
):
    n_trials = int(n_trials)
    # Metric trackers
    coverage_results_EXPONQ = []
    Efficiency_results_EXPONQ = []
    informativeness_results_EXPONQ = []
    coverage_results_PCOQS = []
    Efficiency_results_PCOQS = []
    informativeness_results_PCOQS = []
    coverage_results_np = []
    Efficiency_results_np = []
    informativeness_results_np = []
    accuracy_results = []
    
    # Timing trackers
    time_results_EXPONQ = []
    time_results_PCOQS = []
    time_results_np = []

    for i in tqdm(range(n_trials)):
        # Data generation and splitting 
        X, y = simulate_normal_classification(
            n_samples_per_class=n_samples_per_class,
            n_features=n_features,
            n_classes=n_classes,
            class_means=class_means,
            class_covariances=class_covariances,
            random_state=i + 3
        )
        X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=i + 3)
        X_cal, X_test, y_cal, y_test = train_test_split(X_temp, y_temp, test_size=0.4, random_state=i + 3)
        num_calib = len(X_cal)

        # Train model 
        if private_model:
            model = get_private_model(model_name, X_test, y_train, y, epsilon)
        else:
            model = get_nonprivate_model(model_name, random_state=42)
        model.fit(X_train, y_train)

        # Compute calibration scores (preparation step)
        prob_cal = model.predict_proba(X_cal)
        scores_cal = np.round(1 - prob_cal[np.arange(len(y_cal)), y_cal], 4)

        # Threshold and time computation for each method
        if private_conformal:
            # 1. EXPONQ method
            start_time = time.time()
            mstar, gammastar = get_optimal_gamma_m(num_calib, alpha, conformal_epsilon)
            m = mstar
            gamma = gammastar
            score_bins = np.linspace(0, 1, m)
            shat = get_shat_from_scores_private(scores_cal, alpha, conformal_epsilon, gamma, score_bins)
            threshold_EXPONQ = shat
            time_EXPONQ = time.time() - start_time
            time_results_EXPONQ.append(time_EXPONQ)

            # 2. P-COQS method
            start_time = time.time()
            scores_cal_sorted = np.sort(scores_cal)  
            threshold_PCOQS = PrivQuant(scores_cal_sorted, alpha, conformal_epsilon)
            time_PCOQS = time.time() - start_time
            time_results_PCOQS.append(time_PCOQS)
        else:
            # 3. non-private method
            start_time = time.time()
            threshold = np.quantile(scores_cal, math.ceil((1 - alpha) * (len(scores_cal) + 1)) / len(scores_cal))
            time_np = time.time() - start_time
            time_results_np.append(time_np)

        # Compute prediction sets
        prob_test = model.predict_proba(X_test)
        scores_test = 1 - prob_test

        if private_conformal:
            # Prediction sets for EXPONQ
            prediction_sets_EXPONQ = [
                np.where(scores <= threshold_EXPONQ)[0] for scores in scores_test
            ]
            prediction_sets_EXPONQ = [
                pset if len(pset) > 0 else [-1] for pset in prediction_sets_EXPONQ
            ]
            # Compute informativeness for EXPONQ
            informativeness_EXPONQ = np.mean([len(pset) == 1 for pset in prediction_sets_EXPONQ])
            informativeness_results_EXPONQ.append(informativeness_EXPONQ)

            # Prediction sets for PCOQS
            prediction_sets_PCOQS = [
                np.where(scores <= threshold_PCOQS)[0] for scores in scores_test
            ]
            prediction_sets_PCOQS = [
                pset if len(pset) > 0 else [-1] for pset in prediction_sets_PCOQS
            ]
            # Compute informativeness for PCOQS
            informativeness_PCOQS = np.mean([len(pset) == 1 for pset in prediction_sets_PCOQS])
            informativeness_results_PCOQS.append(informativeness_PCOQS)
        else:
            # Non-private prediction sets
            prediction_sets = [
                np.where(scores <= threshold)[0] for scores in scores_test
            ]
            prediction_sets = [
                pset if len(pset) > 0 else [-1] for pset in prediction_sets
            ]
            # Compute informativeness for non-private
            informativeness = np.mean([len(pset) == 1 for pset in prediction_sets])
            informativeness_results_np.append(informativeness)

        # Compute metrics
        if private_conformal:
            # EXPONQ
            coverage_EXPONQ = np.mean([y_test[i] in prediction_sets_EXPONQ[i] for i in range(len(y_test))])
            coverage_results_EXPONQ.append(coverage_EXPONQ)
            Efficiency_EXPONQ = np.mean([len(pset) for pset in prediction_sets_EXPONQ])
            Efficiency_results_EXPONQ.append(Efficiency_EXPONQ)

            # PrivQuant
            coverage_PCOQS = np.mean([y_test[i] in prediction_sets_PCOQS[i] for i in range(len(y_test))])
            coverage_results_PCOQS.append(coverage_PCOQS)
            Efficiency_PCOQS = np.mean([len(pset) for pset in prediction_sets_PCOQS])
            Efficiency_results_PCOQS.append(Efficiency_PCOQS)
        else:
            # Non-private
            coverage = np.mean([y_test[i] in prediction_sets[i] for i in range(len(y_test))])
            coverage_results_np.append(coverage)
            Efficiency = np.mean([len(pset) for pset in prediction_sets])
            Efficiency_results_np.append(Efficiency)

        # Accuracy
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracy_results.append(accuracy)

    # Aggregate results (add timing metrics)
    results = {}
    if private_conformal:
        results["EXPONQ"] = {
            "coverage_mean": np.mean(coverage_results_EXPONQ),
            "coverage_std": np.std(coverage_results_EXPONQ, ddof=1),
            "Efficiency_mean": np.mean(Efficiency_results_EXPONQ),
            "Efficiency_std": np.std(Efficiency_results_EXPONQ, ddof=1),
            "informativeness_mean": np.mean(informativeness_results_EXPONQ),
            "informativeness_std": np.std(informativeness_results_EXPONQ, ddof=1),
            "time_mean": np.mean(time_results_EXPONQ),
            "time_std": np.std(time_results_EXPONQ, ddof=1),
        }
        results["PCOQS"] = {
            "coverage_mean": np.mean(coverage_results_PCOQS),
            "coverage_std": np.std(coverage_results_PCOQS, ddof=1),
            "Efficiency_mean": np.mean(Efficiency_results_PCOQS),
            "Efficiency_std": np.std(Efficiency_results_PCOQS, ddof=1),
            "informativeness_mean": np.mean(informativeness_results_PCOQS),
            "informativeness_std": np.std(informativeness_results_PCOQS, ddof=1),
            "time_mean": np.mean(time_results_PCOQS),
            "time_std": np.std(time_results_PCOQS, ddof=1),
        }
    else:
        results["nonprivate"] = {
            "coverage_mean": np.mean(coverage_results_np),
            "coverage_std": np.std(coverage_results_np, ddof=1),
            "Efficiency_mean": np.mean(Efficiency_results_np),
            "Efficiency_std": np.std(Efficiency_results_np, ddof=1),
            "informativeness_mean": np.mean(informativeness_results_np),
            "informativeness_std": np.std(informativeness_results_np, ddof=1),
            "time_mean": np.mean(time_results_np),
            "time_std": np.std(time_results_np, ddof=1),
        }
    results["accuracy"] = {
        "mean": np.mean(accuracy_results),
        "std": np.std(accuracy_results, ddof=1),
    }

    return results

### Simulation for different models

In [8]:
def complete_model_conformal_simulation(
    model_sets,
    private_models,
    private_conformals,
    n_samples_per_class,
    n_features,
    n_classes,
    class_means,
    class_covariances,
    alpha,
    epsilon,
    conformal_epsilon,
    n_trials
):
    # Results dictionary to store all results
    results = {}

    # Iterate through models, privacy options, and conformal options
    for model_name in tqdm(model_sets):
        model_results = []  # To collect results for each model
        for private_model in private_models:
            for private_conformal in private_conformals:
                # Run simulation for the current configuration
                sim_results = run_simulation(
                    private_model,
                    private_conformal,
                    model_name,
                    n_samples_per_class,
                    n_features,
                    n_classes,
                    class_means,
                    class_covariances,
                    alpha,
                    epsilon,
                    conformal_epsilon,
                    n_trials
                )

                # Extract results based on the conformal configuration
                if private_conformal:
                    # Add results for EXPONQ.
                    model_results.append({
                        "Description": f"{model_name} - Private Model: {private_model}, Private Conformal: {private_conformal} (EXPONQ)",
                        "Coverage": f"{sim_results['EXPONQ']['coverage_mean']:.4f} ± {sim_results['EXPONQ']['coverage_std']:.4f}",
                        "Efficiency": f"{sim_results['EXPONQ']['Efficiency_mean']:.4f} ± {sim_results['EXPONQ']['Efficiency_std']:.4f}",
                        "Informativeness": f"{sim_results['EXPONQ']['informativeness_mean']:.4f} ± {sim_results['EXPONQ']['informativeness_std']:.4f}",
                        "Time (s)": f"{sim_results['EXPONQ']['time_mean']:.6f} ± {sim_results['EXPONQ']['time_std']:.6f}",
                        "Accuracy": f"{sim_results['accuracy']['mean']:.4f} ± {sim_results['accuracy']['std']:.4f}",
                    })
                    # Add results for PCOQS
                    model_results.append({
                        "Description": f"{model_name} - Private Model: {private_model}, Private Conformal: {private_conformal} (PCOQS)",
                        "Coverage": f"{sim_results['PCOQS']['coverage_mean']:.4f} ± {sim_results['PCOQS']['coverage_std']:.4f}",
                        "Efficiency": f"{sim_results['PCOQS']['Efficiency_mean']:.4f} ± {sim_results['PCOQS']['Efficiency_std']:.4f}",
                        "Informativeness": f"{sim_results['PCOQS']['informativeness_mean']:.4f} ± {sim_results['PCOQS']['informativeness_std']:.4f}",
                        "Time (s)": f"{sim_results['PCOQS']['time_mean']:.6f} ± {sim_results['PCOQS']['time_std']:.6f}",
                        "Accuracy": f"{sim_results['accuracy']['mean']:.4f} ± {sim_results['accuracy']['std']:.4f}",
                    })
                else:
                    # Add results for non-private conformal
                    model_results.append({
                        "Description": f"{model_name} - Private Model: {private_model}, Private Conformal: {private_conformal}",
                        "Coverage": f"{sim_results['nonprivate']['coverage_mean']:.4f} ± {sim_results['nonprivate']['coverage_std']:.4f}",
                        "Efficiency": f"{sim_results['nonprivate']['Efficiency_mean']:.4f} ± {sim_results['nonprivate']['Efficiency_std']:.4f}",
                        "Informativeness": f"{sim_results['nonprivate']['informativeness_mean']:.4f} ± {sim_results['nonprivate']['informativeness_std']:.4f}",
                        "Time (s)": f"{sim_results['nonprivate']['time_mean']:.6f} ± {sim_results['nonprivate']['time_std']:.6f}",
                        "Accuracy": f"{sim_results['accuracy']['mean']:.4f} ± {sim_results['accuracy']['std']:.4f}",
                    })

        # Store results for the current model
        results[model_name] = model_results

    # Generate and print results in a structured format
    for model_name, model_results in results.items():
        print(f"\n{'#' * 10} {model_name} {'#' * 10}\n")
        print(f"{'Description':<70} {'Coverage':<20} {'Efficiency':<20} {'Informativeness':<20} {'Time (s)':<20} {'Accuracy':<20}")
        print("-" * 170)
        for result in model_results:
            print(f"{result['Description']:<70} {result['Coverage']:<20} {result['Efficiency']:<20} {result['Informativeness']:<20} {result['Time (s)']:<20} {result['Accuracy']:<20}")
        print("\n")
    
    return results

### Simulation over different alpha values

In [9]:
def compare_conformal_alpha(
    model_sets,
    epsilon,
    conformal_epsilon,
    private_models,
    private_conformals,
    n_samples_per_class,
    n_features,
    n_classes,
    class_means,
    class_covariances,
    alpha_values,
    n_trials
):
    results = []  # Use a list to store the results
    for alpha in alpha_values:
        print("alpha_value:", alpha)
        #rho = conformal_epsilon
        output = complete_model_conformal_simulation(
                                                    model_sets,
                                                    private_models,
                                                    private_conformals,
                                                    n_samples_per_class,
                                                    n_features,
                                                    n_classes,
                                                    class_means,
                                                    class_covariances,
                                                    alpha,
                                                    epsilon,
                                                    conformal_epsilon,
                                                    n_trials
                                                    )
        # Append results for the current alpha value
        results.append({"alpha": alpha, "output": output})
    return results


### Running the simulation

In [10]:
model_sets = ["Naive_bayes", "Random_forest"]
private_models = [True, False]
private_conformals = [True, False]
n_samples_per_class = 5000
n_features = 8
n_classes = 2
class_means = [
    np.array([0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8]),
    np.array([-1, -1, -1, -1, -1, -1, -1, -1])
]
class_covariances = [
    np.eye(8) * 7,
    np.eye(8) * 8
]
n_trials = 1000
alpha_values = [0.1]
epsilon = 2
conformal_epsilon = 1

# Save/load logic
results_filename = "conformal_results_computational_time.pkl"

if os.path.exists(results_filename):
    print("Loading saved results...")
    with open(results_filename, "rb") as f:
        results = pickle.load(f)
else:
    print("Running experiments...")
    results = compare_conformal_alpha(
        model_sets, epsilon, conformal_epsilon, private_models, private_conformals,
        n_samples_per_class, n_features, n_classes, class_means, class_covariances,
        alpha_values, n_trials
    )
    with open(results_filename, "wb") as f:
        pickle.dump(results, f)
    print(f"Results saved to {results_filename}")

Running experiments...
alpha_value: 0.1


100%|███████████████████████████████████████| 1000/1000 [22:48<00:00,  1.37s/it]

100%|███████████████████████████████████████| 1000/1000 [00:52<00:00, 19.05it/s]

100%|███████████████████████████████████████| 1000/1000 [22:20<00:00,  1.34s/it]

100%|███████████████████████████████████████| 1000/1000 [38:45<00:00,  2.33s/it]

100%|███████████████████████████████████████| 1000/1000 [22:05<00:00,  1.33s/it]

100%|███████████████████████████████████████| 1000/1000 [38:48<00:00,  2.33s/it]

100%|█████████████████████████████████████████| 2/2 [2:45:45<00:00, 4972.74s/it]


########## Naive_bayes ##########

Description                                                            Coverage             Efficiency           Informativeness      Time (s)             Accuracy            
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Naive_bayes - Private Model: True, Private Conformal: True (EXPONQ)    0.9217 ± 0.0116      1.4841 ± 0.0457      0.5159 ± 0.0457      1.306383 ± 0.081798  0.7479 ± 0.0125     
Naive_bayes - Private Model: True, Private Conformal: True (PCOQS)     0.8999 ± 0.0098      1.3985 ± 0.0322      0.6015 ± 0.0322      0.010917 ± 0.003472  0.7479 ± 0.0125     
Naive_bayes - Private Model: True, Private Conformal: False            0.8998 ± 0.0096      1.3981 ± 0.0318      0.6019 ± 0.0318      0.000210 ± 0.000212  0.7479 ± 0.0125     
Naive_bayes - Private Model: False, Private Conformal: True (EXPONQ)   0.9227 ± 0.0116   

In [11]:
results

[{'alpha': 0.1,
  'output': {'Naive_bayes': [{'Description': 'Naive_bayes - Private Model: True, Private Conformal: True (EXPONQ)',
     'Coverage': '0.9217 ± 0.0116',
     'Efficiency': '1.4841 ± 0.0457',
     'Informativeness': '0.5159 ± 0.0457',
     'Time (s)': '1.306383 ± 0.081798',
     'Accuracy': '0.7479 ± 0.0125'},
    {'Description': 'Naive_bayes - Private Model: True, Private Conformal: True (PCOQS)',
     'Coverage': '0.8999 ± 0.0098',
     'Efficiency': '1.3985 ± 0.0322',
     'Informativeness': '0.6015 ± 0.0322',
     'Time (s)': '0.010917 ± 0.003472',
     'Accuracy': '0.7479 ± 0.0125'},
    {'Description': 'Naive_bayes - Private Model: True, Private Conformal: False',
     'Coverage': '0.8998 ± 0.0096',
     'Efficiency': '1.3981 ± 0.0318',
     'Informativeness': '0.6019 ± 0.0318',
     'Time (s)': '0.000210 ± 0.000212',
     'Accuracy': '0.7479 ± 0.0125'},
    {'Description': 'Naive_bayes - Private Model: False, Private Conformal: True (EXPONQ)',
     'Coverage': '0.9

### Processing results for tabular presentation

In [12]:
def generate_formatted_tables(results):
    """Generate tables with Description, Coverage, Efficiency, Informativeness, Accuracy, and Time."""
    # Extract and filter data
    filtered_data = []
    for entry in results:
        for model, model_data in entry['output'].items():
            for config in model_data:
                if "Private Conformal: True" in config['Description']:
                    filtered_data.append({
                        'Description': config['Description'],
                        'Coverage': config['Coverage'],
                        'Efficiency': config['Efficiency'],
                        'Informativeness': config['Informativeness'],
                        'Model Accuracy': config['Accuracy'],
                        'Ave. time (secs)': config['Time (s)']
                    })

    # Create DataFrame and clean up descriptions
    df = pd.DataFrame(filtered_data)
    
    # Format the Description column to match your example
    df['Description'] = df['Description'].apply(lambda x: 
        "f_DP and ExponQ" if "Private Model: True" in x and "EXPONQ" in x else
        "f_DP and P-COQS" if "Private Model: True" in x and "PCOQS" in x else
        "f and ExponQ" if "Private Model: False" in x and "EXPONQ" in x else
        "f and P-COQS" if "Private Model: False" in x and "PCOQS" in x else x)
    
    # Reorder columns
    df = df[['Description', 'Coverage', 'Efficiency', 'Informativeness', 
             'Model Accuracy', 'Ave. time (secs)']]
    
    return df

# Generate and print the table
result_table = generate_formatted_tables(results)
print(result_table.to_string(index=False))

    Description        Coverage      Efficiency Informativeness  Model Accuracy    Ave. time (secs)
f_DP and ExponQ 0.9217 ± 0.0116 1.4841 ± 0.0457 0.5159 ± 0.0457 0.7479 ± 0.0125 1.306383 ± 0.081798
f_DP and P-COQS 0.8999 ± 0.0098 1.3985 ± 0.0322 0.6015 ± 0.0322 0.7479 ± 0.0125 0.010917 ± 0.003472
   f and ExponQ 0.9227 ± 0.0116 1.2509 ± 0.0360 0.7491 ± 0.0360 0.8253 ± 0.0092 1.291798 ± 0.098815
   f and P-COQS 0.9006 ± 0.0099 1.1788 ± 0.0201 0.8212 ± 0.0201 0.8253 ± 0.0092 0.011768 ± 0.004063
f_DP and ExponQ 0.9242 ± 0.0123 1.3459 ± 0.0555 0.6541 ± 0.0555 0.7941 ± 0.0173 1.098612 ± 0.064621
f_DP and P-COQS 0.8977 ± 0.0106 1.2533 ± 0.0438 0.7467 ± 0.0438 0.7941 ± 0.0173 0.010933 ± 0.003062
   f and ExponQ 0.9229 ± 0.0120 1.2939 ± 0.0406 0.7061 ± 0.0406 0.8125 ± 0.0093 1.131960 ± 0.084846
   f and P-COQS 0.9025 ± 0.0098 1.2223 ± 0.0224 0.7777 ± 0.0224 0.8125 ± 0.0093 0.011055 ± 0.003292
